# Ownership（所有权）

In [ ]:
let julio = Bear::get();
// let ryan = julio;

函数中的**变量**或者**结构体成员**可以看成*所有者* Owner, 而右边的东西（属于所有者的）可以看成*数据*或者*值*

上面的代码进行可视化，如图所示。

<img src="./asset/ownership/ownership-bear-1.png", width="800"/> ![ownership visualized](./asset/ownership/ownership-bear-1.png)

当所有者玩完玩具后，需要将玩具放回原处。

In [ ]:
fn main() {
    let julio = Bear::get();
    // play with bear
}

如图所示：

![ownership visualized](./asset/ownership/ownership-bear-2.png)

`julio` 是玩具熊的所有者，他可以对玩具熊做任何事情，同时他也需要对玩具熊负责，当玩完玩具熊后，或者需要离开的时候，需要将玩具熊放回原处（内存释放）。

In [ ]:
let julio = Bear::get();
let ryan = julio;

上面的代码可以看成是：本来玩具熊 🧸 的所有者是 `julio`, 然后因为第二行的代码，🧸 的所有者变成了 `ryan`。`ryan` 可以对玩具熊 🧸 做任何事情，`ryan` 需要对玩具熊负责，在 `ryan` 离开时需要将玩具熊放回原处（在那里发现它的）

因为`julio`已经将玩具熊给了`ryan`，所以他此时已经无法对玩具熊做任何事情了，同时他也不用为玩具熊负任何责任了。


## 什么时候会发生所有权的转移呢？


1. 函数调用

- 函数调用可以获取变量的所有权（函数调用可以对玩具熊进行任何操作，因为是所有者了）
- 同时也意味着在函数调用返回时，需要对变量负责，「释放变量的内存」（在函数调用离开时，需要将玩具熊放回原处，内存释放）
- 当函数返回后，你不再能玩玩具熊 🧸 了（因为在函数调用返回后，玩具熊已经被放回原处了，此时已经不在拥有玩具熊了）

In [ ]:
let julio = Bear::get();

my_cool_bear_function(julio); // 这一步会让这个函数拥有玩具熊🧸，也就是说这个函数是玩具熊🧸的所有者

// 函数调用后，julio 不再拥有玩具熊了

2. 变量之间
```rust
my_cool_bear_function(/* parameter */) {
    // Do stuff
}
```

可以这样思考 `parameter`

- 他是一个函数中的局部变量
- 他拥有数据（他是玩具熊的所有者）
- 他需要对玩具熊负责，在他离开作用域时，需要将玩具熊放回原处（释放玩具熊的内存）
- 一旦数据被清除，将不会再被使用了（一旦把玩具熊放回原处，就不能继续玩玩具熊了）

`julio` 将玩具熊给了别人，在函数完成时，那个人就要“回家”，同时他需要对玩具熊负责，将玩具熊放回原处，一旦玩具熊🧸被放回原处，`julio` 就不能和玩具熊玩耍啦

## Borrowing

In [ ]:
let julio = Bear::get();

my_cool_bear_function(&julio) // 注意参数中的 &

/* julio 变量在函数调用之后还可以和玩具熊玩耍
（他还是玩具熊的所有者，在函数调用时，只是将玩具熊借出去了而已） */

如图所示：

![borrowing](./asset/ownership-bear-3.png)

**所有权的规则**

- Rust中每一个值/（数据）都有一个被称为所有者的变量
- 同一时间，只能有一个所有者
- 当所有者超出范围时，该值（数据）将被丢弃（drop）

## 所有权在内存中长什么样？

比如如下的示例

In [ ]:
let julio = "Hi, friends".to_string();

`"Hi, friends".to_string()` 将会被分配到堆上，同时在栈上保存了字符串的相关信息，比如字符串的长度、容量、和指针。

如图所示：

![memory](./asset/ownership/memory-1.png)

In [ ]:
let julio = "Hi, friends".to_string();
let ryan = julio;

此时，值/数据 的所有权就会发生转移，数据的所有者变成了 `ryan`。

在其他语言中，这被称为浅拷贝，只是复制了栈上面的空间，而堆上面的空间仍然指向了同一个地方。

如图所示

![memory](./asset/ownership/memory-2.png)

但是在 Rust 中，`julio` 将不会是数据的所有者了，因此无法对数据进行任何操作，这样避免了二次释放内存等问题。

如图所示

![memory](./asset/ownership/memory-3.png)

In [4]:
fn mian() {
    let julio = "Hi, friends".to_string();
    let ryan = julio;
    // println!("{}", ryan);
}

在变量离开它的作用域时，`Drop` 函数将被调用，丢弃拥有值（数据）的变量

- 当到达作用域 [`{}`内] 的末尾时，Drop 函数就会被调用
- 可以认为是一种特殊的函数，以正确释放整个对象（可能有多个指针需要释放，因此该函数具有该实现）
- 具有 Drop trait 的类型拥有一个 Drop 函数可供调用

如果没有阻止`julio`访问复制的 string 数据会发生什么呢？可能会进行二次释放 D:

如图所示

![double free](./asset/ownership/memory-4.png)


- 在传递所有权时，对变量进行了浅拷贝，然后使之前的无效的变量不再拥有那个数据
- 这个无效的操作避免了二次内存释放，此外还知道其实是调用了 Drop 函数
- 如果想要做一次深拷贝（在堆上创建一个新的相同的数据），Rust 有 `clone` 函数

### clone function

In [ ]:
let julio = "Hi, friends".to_string();
let ryan = julio.clone();

上面代码的效果就是相当于将制造了一份与`julio`一样的数据，然后把它给到`ryan`

如图所示

![clone](./asset/ownership/clone.png)

那么下面的代码堆区间会发生什么呢？

如图所示

![stack-1](./asset/ownership/stack-1.png)


In [ ]:
let julio = 10;
let ryan = julio;

如果我们不去阻止 `julio` 访问 number 对象中副本的值，会发生什么呢？

如图所示

![stack](./asset/ownership/stack-2.png)
>因为数字类型大小是确定的，只在栈分配即可，不涉及到堆空间

- rust 中一些值不使用堆空间，会直接分配到栈上面（integer types, bool, etc...）
    - 对于这些类型，一个“浅拷贝”就是一个完全的复制
- 只需要栈空间的对象通常在变量赋值时默认进行复制
    - 这些类型有 **Copy** trait
        - ‘=’操作将会创建一个 copy，而不是转移所有权
- 如果有一个 *Copy* trait，Rust 将不会让你实现 *Drop* trait

如果没有 copy trait，rust将会认为是移动所有权

### Borrowong++
> & 背后的含义是什么？

 **Rust中变量规则**

 - Rust中，默认情况下，所有数据都是不可变的
 - 你可以想象，每个实例化的变量的背后都暗藏着 `const`
 - `mut` 关键字指定变量拥有的数据可变，它就像 `const` 的相反
 - 如果更改并未声明为`mute`的变量的数据值，Rust编译器将不会编译代码

可变变量如图所示

![mutable variables](./asset/ownership/mut-var.png)


### Borrowing 创造了一个类型

如图所示

![borrow](./asset/ownership/borrow-1.png)

### borrowing type == reference (借用的类型 == 引用)

- `&` 创造了一个新的变量类型，称为对该类型的引用
- 因为是新的变量，所以他们默认也是不可变的，他们可以被声明为可变的
- 只有实际的变量是可变的时候，才能创建可变引用

以下是不可变引用和可变引用的示例

In [ ]:
let julio = Bear::get();
let julio_reference = &julio;

my_cool_bear_function(julio_reference);

/* julio 变量可以在这里被使用 */

In [ ]:
let mut julio = Bear::get();
let mutable_julio_reference = &mut julio;

my_cool_bear_function(mutable_julio_reference);

/* julio 变量可以在这里被使用 */

**不可变引用**
> 下面的代码不会被编译通过

In [ ]:
// append_to_vector 函数接收一个向量的引用作为参数
fn append_to_vector(lst: &Vec<u32>) {
    lst.push(3);
}

// main 函数传递一个引用到 append_to_vector
fn main() {
    let mut lst = vec![1, 2, 3];
    append_to_vector(&let)
}

**可变引用**
> 编译通过

In [ ]:
// append_to_vector 函数接收一个向量的引用作为参数， 注意 &mut
// ⚠️ 必须是可变引用，因为向量被改变了！
fn append_to_vector(lst: &mut Vec<u32>) {
    lst.push(3);
}

// main 函数传递一个 *可变* 引用到 append_to_vector
fn main() {
    let mut lst = vec![1, 2, 3];
    append_to_vector(&mut let);
}

### Borrowing + References: 陷阱！

我们希望两个画家都相信，他们看到的熊仔她们尝试绘画时不会改变！

如图所示

![borrow_reference](./asset/ownership/borrow-ref-1.png)

In [ ]:
let mut bear = Bear::get();
let pink_shirt = &bear;
let blue_shirt = &bear;

如果这时候有一个坏的派大星，获得了对小熊的可变引用，那么就会出现问题

如图所示

![borrow_mut_ref](./asset/ownership/borrow-ref-2.png)

In [ ]:
let mut bear = Bear::get();
let pink_shirt = &bear;
let blue_shirt = &bear;
let evil_patrick = &mut bear;

### 引用规则

- 在一个作用域内，一个变量可以有多个不可变引用。
    - 很多画家只要知道在他们作画时没有任何人会改变那头熊，就能够画出一幅熊的图片。
- 但是在一个作用域内，只能有一个可变引用
    - 否则，不可变引用可能会观察到与其最初期望的数据不同的数据，或者两个可变引用的更改可能会发生冲突

    > 要不就是可以有多个不可变引用，不要就是只能有一个可变引用，而且不能存在不可变引用

    如图所示

    ![references rules](./asset/ownership/ref-rule-1.png)
- 注意：如果创造了一个引用，那么原始的变量会有以下情况：
    - 如果这个引用是可变的：那么原始变量暂时无法使用
    - 如果这个引用是不可变的：那么原始变量暂时无法更改

>这样可以保证数据的一致性：
>1. 如果有尝试写，则别的变量无法读取数据
>2. 如果有尝试读，则别的变量无法修改数据

**代码示例**

*避免迭代器失效*

In [ ]:
fn main() {
    let mut v = vec![1, 2, 3];
    /* for loop 借用了上面的向量完成工作 */
    for i in &mut v {
        println!("{}", i);
        v.push(34); /* 可能导致大小调整 -> 移动到新的内存 */
    }
}

如图所示：

![iter-invalid](./asset/ownership/iter-invalid.png)

## 引用总结

- 通过所有权和借用规则，可以避免许多不同类型的内存错误 :D
- 但是它们确实会导致更棘手的代码编写 -- Rust编译器将会在人写程序时进行斗争